In [ ]:
!mkdir /content/sample_data/project
%cd /content/sample_data/project
!mkdir test
!mkdir train
!mkdir val

/content/sample_data/project


In [ ]:
!unzip /content/sample_data/project/train/images-train.zip -d /content/sample_data/project/train

Archive:  /content/sample_data/project/train/images-train.zip
   creating: /content/sample_data/project/train/images/
  inflating: /content/sample_data/project/train/images/image_1.png  
  inflating: /content/sample_data/project/train/images/image_10.png  
  inflating: /content/sample_data/project/train/images/image_100.png  
  inflating: /content/sample_data/project/train/images/image_101.png  
  inflating: /content/sample_data/project/train/images/image_102.png  
  inflating: /content/sample_data/project/train/images/image_103.png  
  inflating: /content/sample_data/project/train/images/image_104.png  
  inflating: /content/sample_data/project/train/images/image_105.png  
  inflating: /content/sample_data/project/train/images/image_106.png  
  inflating: /content/sample_data/project/train/images/image_107.png  
  inflating: /content/sample_data/project/train/images/image_108.png  
  inflating: /content/sample_data/project/train/images/image_109.png  
  inflating: /content/sample_data

In [ ]:
!unzip /content/sample_data/project/test/images-test.zip -d /content/sample_data/project/test

Archive:  /content/sample_data/project/test/images-test.zip
   creating: /content/sample_data/project/test/images/
  inflating: /content/sample_data/project/test/images/image_251.png  
  inflating: /content/sample_data/project/test/images/image_252.png  
  inflating: /content/sample_data/project/test/images/image_253.png  
  inflating: /content/sample_data/project/test/images/image_254.png  
  inflating: /content/sample_data/project/test/images/image_255.png  
  inflating: /content/sample_data/project/test/images/image_256.png  
  inflating: /content/sample_data/project/test/images/image_257.png  
  inflating: /content/sample_data/project/test/images/image_258.png  
  inflating: /content/sample_data/project/test/images/image_259.png  
  inflating: /content/sample_data/project/test/images/image_260.png  
  inflating: /content/sample_data/project/test/images/image_261.png  
  inflating: /content/sample_data/project/test/images/image_262.png  
  inflating: /content/sample_data/project/tes

In [ ]:
!unzip /content/sample_data/project/val/images-val.zip -d /content/sample_data/project/val

# From google gemini

In [ ]:
!pip install  datasets accelerate evaluate
#transformers
#torch

In [ ]:
import os
from PIL import Image

def create_dataset_dict(data_path: str):#, img_dir: str):
    """
    Parses a text file where each entry begins with a key (e.g., 'image.png')
    followed by a tab, and then a paragraph of text that can span multiple lines.
    It collects all text until the next key is found, and converts the data
    into a JSON file.

    Args:
        input_filepath: The path to the source text file.
        output_filepath: The path where the output JSON file will be saved.
    """
    #data_dict = {}
    data_dict = []
    current_key = None
    current_value_lines = []
    labels_path = os.path.join(data_path, 'labels.txt')
    images_path = os.path.join(data_path, 'images')

    try:
        if not os.path.exists(labels_path):
            print(f"Error: The input file '{labels_path}' was not found.")
            return

        print(f"Reading data from '{labels_path}'...")
        with open(labels_path, 'r', encoding='utf-8') as infile:
            for line in infile:
                # Strip leading and trailing whitespace from the line
                clean_line = line.strip()

                # Check if the line is a new key-value entry
                # A new entry is identified by a tab character and a key ending in .png
                if '\t' in clean_line and clean_line.split('\t')[0].endswith('.png'):
                    # If we have a previous key, save the collected value to the dictionary
                    if current_key:
                        data_dict.append({
                            "image": os.path.join(images_path, current_key),
                            "text": "\n".join(current_value_lines).strip()
                        })
                    # Start a new key-value pair
                    parts = clean_line.split('\t', 1)
                    current_key = parts[0]
                    current_value_lines = [parts[1]]

                # If the line is not a new key, it's a continuation of the current paragraph
                elif current_key:
                    current_value_lines.append(clean_line)

                # If there's an empty line at the beginning of the file before any key
                # just ignore it.
                elif not clean_line:
                    continue
                else:
                    print(f"Warning: Skipping line with unexpected format: '{clean_line}'")
            # After the loop, save the very last key-value pair
            if current_key:
                data_dict.append({
                    "image": os.path.join(images_path, current_key),
                    "text": "\n".join(current_value_lines).strip()
                })
        print(f"Successfully parsed {len(data_dict)} entries.")
        return data_dict
    except IOError as e:
        print(f"An I/O error occurred: {e}")
        return []
    except Exception as e:
        print(f"An unexpected error occurred: {e}")
        return []

In [ ]:
import os
os.chdir("/content/sample_data/project") # Change to the project directory
# Create the training and testing lists
train_data_list = create_dataset_dict('train/')
test_data_list = create_dataset_dict('test/')

print(f"Number of training samples: {len(train_data_list)}")
print(f"Number of testing samples: {len(test_data_list)}")

# Print a sample to verify the data structure
if train_data_list:
    print("\nSample training data entry:")
    print(train_data_list[0])
    print(train_data_list[1])


Reading data from 'train/labels.txt'...
Successfully parsed 200 entries.
Reading data from 'test/labels.txt'...
Successfully parsed 50 entries.
Number of training samples: 200
Number of testing samples: 50

Sample training data entry:
{'image': 'train/images/image_1.png', 'text': 'الجريدة الرسمية - العدد ١ - ٢٠٢٥/١/٢\nالمجلس الدستوري\nقرار رقم: ١٢ / ٢٠٢٤\nرقم المراجعة ١٤ / و\nتاريخ الورود : ٢٠٢٤/١٢/١٩\nموضوع المراجعة القانون رقم ٢٠٢٤/٣٢٨\nورقم ۱۱۰۹۰ تاریخ ۲۷ / ۲ / ۲۰۲۳\nورقم ١٢١٥٩ تاریخ ٢٠٢٣/٩/١١\nورقم ۱۲۸۵۷ تاریخ ٢٠٢٤/۱/۱۲\nإن مجلس الوزراء\nبناء على الدستور ولا سيما المادة ٦٢ منه\nالصادر في ۲۰۱٤/۱۱/۲۸ (تعليق المهل القانونية بناء على قانون المحاسبة العمومية وتعديلاته، لاسيما\nوالقضائية والعقدية المنشور في ملحق العدد /٤٩/ من المادة ٥٢ منه\nالجريدة الرسمية تاريخ ٢٠٢٤/١٢/٥.\nبناء على المرسوم رقم ۱۰۹۸۲ تاریخ ۲۰۲۳/۲/۶\nإن المجلس الدستوري الملتئم في مقره عند (إعطاء الهيئة العليا للإغاثة سلفة خزينة)\nالساعة الحادية عشرة من يوم الاثنين الواقع فيه\n۲۰۲٤/١٢/٢٣ ، برئاسة رئيسه القاضي طنوس مشلب\nبن

In [ ]:
# 1. Choose a Pre-trained Model
# A good choice for this task would be an encoder-decoder model like TrOCR (Transformer-based
# Optical Character Recognition) or a more specialized Arabic model if available.
from transformers import TrOCRProcessor, VisionEncoderDecoderModel

model_name = "microsoft/trocr-base-handwritten" # A base model to fine-tune
processor = TrOCRProcessor.from_pretrained(model_name)
model = VisionEncoderDecoderModel.from_pretrained(model_name)

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(
Using a slow image processor as `use_fast` is unset and a slow processor was saved with this model. `use_fast=True` will be the default behavior in v4.52, even if the model was saved with a slow processor. This will result in minor differences in outputs. You'll still be able to use a slow processor with `use_fast=False`.
Some weights of VisionEncoderDecoderModel were not initialized from the model checkpoint at microsoft/trocr-base-handwritten and are newly initialized: ['enco

In [ ]:
# ---- Step 2: Freeze most encoder layers ----
for param in model.encoder.parameters():
    param.requires_grad = False
# (Optional) Unfreeze last 2 layers of the encoder
for layer in model.encoder.encoder.layer[-2:]:
    for param in layer.parameters():
        param.requires_grad = True

In [ ]:
import torch
from PIL import Image
from transformers import TrOCRProcessor, VisionEncoderDecoderModel
from torch.utils.data import Dataset, DataLoader
#from datasets import load_metric
import evaluate
from transformers import Seq2SeqTrainer, Seq2SeqTrainingArguments

In [ ]:
# 2. Preprocess the Data
# The images and text need to be converted into a format the model can understand.
# This involves tokenizing the text and resizing/normalizing the images.
# You would create a custom Dataset class for PyTorch that handles this on-the-fly.

class OCRDataset(Dataset):
     def __init__(self, data_list, processor):
         self.data_list = data_list
         self.processor = processor
     def __len__(self):
         return len(self.data_list)
     def __getitem__(self, idx):
         item = self.data_list[idx]
         print(item)
         #image_path = os.path.join(list(item.values())[0], 'images') #list(item.values())[0]
         image = Image.open(item['image']).convert("RGB")
         text = item['text']

         # Tokenize the text and prepare the image
         labels = self.processor.tokenizer(text, padding="max_length", truncation=True, return_tensors="pt").input_ids
         pixel_values = self.processor(images=image, return_tensors="pt").pixel_values
         return {"pixel_values": pixel_values.squeeze(), "labels": labels.squeeze()}


In [ ]:
# Instantiate the datasets
train_dataset = OCRDataset(train_data_list, processor)
test_dataset = OCRDataset(test_data_list, processor)

# Set the decoder start token and pad token
model.config.decoder_start_token_id = processor.tokenizer.cls_token_id
model.config.pad_token_id = processor.tokenizer.pad_token_id

# Set the max length for the decoder
model.config.max_length = 512

# Disable cache for the model during fine-tuning
model.config.use_cache = False

In [ ]:
def compute_metrics(pred):
    labels_ids = pred.label_ids
    pred_ids = pred.predictions

    # Replace -100 with the pad token id to decode
    labels_ids[labels_ids == -100] = processor.tokenizer.pad_token_id

    # Decode the predictions and labels
    pred_str = processor.tokenizer.batch_decode(pred_ids, skip_special_tokens=True)
    label_str = processor.tokenizer.batch_decode(labels_ids, skip_special_tokens=True)

    # Compute CER
    cer = cer_metric.compute(predictions=pred_str, references=label_str)

    return {"cer": cer}

In [ ]:
!pip install jiwer

In [ ]:
# --- Collate function for DataLoader ---
def collate_fn(batch):
    # This function is used to batch the data correctly
    pixel_values = torch.stack([item["pixel_values"] for item in batch])
    labels = torch.stack([item["labels"] for item in batch])
    return {"pixel_values": pixel_values, "labels": labels}

# --- Evaluation Metric ---
# We will use the Character Error Rate (CER) to evaluate our model
cer_metric = evaluate.load("cer")

In [ ]:
# --- Training Arguments ---
# This is where we define our training hyperparameters
training_args = Seq2SeqTrainingArguments(
    output_dir="./trocr-fine-tuned-arabic",
    per_device_train_batch_size=2, # Reduced batch size
    gradient_accumulation_steps=4, # Accumulate gradients over 4 steps
    per_device_eval_batch_size=2, # Reduced batch size
    learning_rate=2e-5,
    num_train_epochs=5,
    eval_strategy="epoch",
    save_strategy="epoch",
    logging_steps=10,
    save_total_limit=2,
    predict_with_generate=True,
    fp16=torch.cuda.is_available(), # Use mixed-precision training if a GPU is available
    push_to_hub=False,
)

# --- Trainer Initialization and Training ---
trainer = Seq2SeqTrainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=test_dataset,
    #tokenizer=processor.feature_extractor, # Pass the feature extractor
    processing_class=processor, # Use processing_class instead of tokenizer
    data_collator=collate_fn,
    compute_metrics=compute_metrics,
)

In [ ]:
torch.cuda.empty_cache()

In [ ]:
!export PYTORCH_CUDA_ALLOC_CONF="expandable_segments:True"

In [ ]:
#Disable Cache:
model.config.use_cache = False  # Ensure this is set to False

In [ ]:
# Start the training process
print("Starting training...")
trainer.train()
print("Training finished.")

# --- Save the fine-tuned model and processor ---
model.save_pretrained("./trocr-fine-tuned-arabic")
processor.save_pretrained("./trocr-fine-tuned-arabic")
print("Model and processor saved.")

Starting training...


wandb: Currently logged in as: katkoutkiki81 (katkoutkiki81-intelli) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


{'image': 'train/images/image_143.png', 'text': 'الجريدة الرسمية - العدد ٣ - ٢٠٢٥/١/١٦\nمصلحة الشؤون المالية، مبنى وزارة الاتصالات، شارع وبعد استشارة مجلس شورى الدولة بموجب الرأي\nالمصارف، بيروت.\nبيروت في ١٣ كانون الثاني ٢٠٢٥\nوزارة البيئة\nقرار رقم 1/1\nوزير الإتصالات\nرقم ٢٠٢٣/١٢٢-٢٠٢٤ تاريخ ٢٠٢٤/٦/١١\nيقرر ما يأتي:\nالمادة الأولى: الغاية من القرار\nم. جوني القرم يهدف هذا القرار الى تحديد إجراءات «عدم\nالإعتراض على مذكرات تصميم المشاريع وطلبات\nواقتراحات تمويل المشاريع المقدمة من الهيئات\nالمعتمدة الى الصندوق الأخضر للمناخ عبر وزارة\nالبيئة للحد من انبعاثات الغازات الدفيئة والتقليص من\nيتعلق بتحديد إجراءات عدم الإعتراض تأثير تغير المناخ تطبيقا لاتفاق باريس الملحق باتفاقية\nالأمم المتحدة الإطارية بشأن تغير المناخ.\nعلى مقترحات التمويل المقدمة الى\nالصندوق الأخضر للمناخ\nإن وزير البيئة\nالمادة الثانية: تحديد الهيئات المعتمدة\nودورها\n1 - ان الهيئات المعتمدة للعمل ضمن نطاق\nبناء على المرسوم رقم ٨٣٧٦ تاریخ ۲۰۲۱/۹/۱۰ الجمهورية اللبنانية هي الهيئات التي وافق عليها\nالصندوق الأخضر للمناخ ب

`loss_type=None` was set in the config but it is unrecognised.Using the default loss: `ForCausalLMLoss`.


Epoch,Training Loss,Validation Loss


{'image': 'train/images/image_196.png', 'text': 'الجريدة الرسمية - العدد ٤ - ٢٠٢٥/١/٢٣\nبيان علم وخبر رقم )\n00\nبتأسيس جمعية بإسم:\nجمعية غنوة لكفالة اليتيم\nمركزها النبطية\nان وزير الداخلية والبلديات\nبناء على المرسوم رقم ٨٣٧٦ تاریخ ۲۰۲۱/۹/۱۰\nبناء على قانون الجمعيات الصادر في 3 آب ۱۹۰۹\nولا سيما المادة السادسة منه\n۳۷۳\nعلى أن تطبق البنود المذكورة أعلاه وفقا\nللقوانين والأنظمة المرعية الإجراء وبعد موافقة\nالمراجع المختصة.\nالمؤسسون السادة\nد. غنوا عبد الكريم صباغ\nد. ساره عمار شميساني\nفاطمه ابراهیم ناصر\nجميله جمال ضاهر\nد. عمار حسن شميساني\nبناء على التعميم رقم ١٠/م/٢٠٠٦ تاريخ\n۲۰۰٦/٥/١٩ وتعديله رقم ١٥/م/۲۰۰۸ تاريخ د. محمد مازن كمال حلاوي\n۲۰۰۸/۹/۱۲ والتعميم رقم ٢٤/م/۲۰۱۸ تاريخ علي حسين جابر\n۲۰۱۸/۱۱/۸\nممثلة الجمعية تجاه الحكومة: الدكتورة غنوا\nبناء على الاعلام المقدم الى وزارة الداخلية والبلديات عبد الكريم صباغ.\nمن مؤسسي الجمعية المسماة «جمعية غنوة لكفالة المادة الثانية على الهيئة التأسيسية إستكمال\nاليتيم والمسجل لدى المديرية الادارية المشتركة اجراءات تأسيس الجمعية والدعوة الى

OutOfMemoryError: CUDA out of memory. Tried to allocate 786.00 MiB. GPU 0 has a total capacity of 14.74 GiB of which 538.12 MiB is free. Process 291798 has 14.21 GiB memory in use. Of the allocated memory 13.24 GiB is allocated by PyTorch, and 857.88 MiB is reserved by PyTorch but unallocated. If reserved but unallocated memory is large try setting PYTORCH_CUDA_ALLOC_CONF=expandable_segments:True to avoid fragmentation.  See documentation for Memory Management  (https://pytorch.org/docs/stable/notes/cuda.html#environment-variables)

Don't run this

In [ ]:

# ---- Step 1: Load processor & model ----
processor = TrOCRProcessor.from_pretrained("microsoft/trocr-small-stage1")
model = VisionEncoderDecoderModel.from_pretrained("microsoft/trocr-small-stage1")

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


preprocessor_config.json:   0%|          | 0.00/276 [00:00<?, ?B/s]

Using a slow image processor as `use_fast` is unset and a slow processor was saved with this model. `use_fast=True` will be the default behavior in v4.52, even if the model was saved with a slow processor. This will result in minor differences in outputs. You'll still be able to use a slow processor with `use_fast=False`.


tokenizer_config.json:   0%|          | 0.00/327 [00:00<?, ?B/s]

sentencepiece.bpe.model:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/238 [00:00<?, ?B/s]

config.json: 0.00B [00:00, ?B/s]

pytorch_model.bin:   0%|          | 0.00/246M [00:00<?, ?B/s]

Some weights of VisionEncoderDecoderModel were not initialized from the model checkpoint at microsoft/trocr-small-stage1 and are newly initialized: ['encoder.pooler.dense.bias', 'encoder.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


generation_config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [ ]:
# ---- Step 4: Data augmentation ----
augment = A.Compose([
    A.RandomRotate90(p=0.3),
    A.Rotate(limit=5, p=0.5),
    A.RandomBrightnessContrast(p=0.5),
    A.MotionBlur(p=0.2),
    A.GaussianBlur(p=0.2),
    A.Resize(384, 384)  # match model input size
])